In [11]:
#!pip install pql
import pql
testAll = pql.find("elemets == all(['Si', 'O', 3])")

In [12]:
from optimade.filter import Parser
from lark import Transformer
p = Parser(version=(0, 9, 5))

In [19]:

x = parseInput(cleandedPQL)
print(x)
pql.find(x)

formula_prototype=="AB2C3" and nelements==4 and elements==all(['Si', ' O'])


{'$and': [{'formula_prototype': 'AB2C3'},
  {'nelements': 4},
  {'elements': {'$all': ['Si', ' O']}}]}

In [65]:
import pql
from optimade.filter import Parser
from lark import Transformer


optiMadeToPQLOperatorSwitch = {
    "=":"==",
    "<=":"<=",
    ">=":">=",
    "=>":">=",
    "!=": "!=",
    "<":"<",
    ">":">",
}

def OptiMadeToPQLOperatorValidator(x):
    return optiMadeToPQLOperatorSwitch[x]

def combineMultiple(string, index):
    firstQuoteIndex = 0
    for firstQuoteIndex in reversed(range(0, index)):
        if(ord(string[firstQuoteIndex]) == 34):
            break
    lastQuoteIndex = index
    for lastQuoteIndex in range(index, len(string)):
        if(ord(string[lastQuoteIndex])== 34):
            break
    insertion = string[firstQuoteIndex+1:lastQuoteIndex]
    insertion = insertion.split(",")
    return string[:firstQuoteIndex] + 'all({})'.format(insertion) + string[lastQuoteIndex+1:]

class OptimadeToPQLTransformer(Transformer):
    def comparison(self, args):
        A = args[0]
        B = args[2]
        operator = OptiMadeToPQLOperatorValidator(args[1])
        return A + operator + B
    def atom(self, args):
        return args[0]
    def term(self, args):
        result = ""
        for arg in args:
            result = result + " " + arg
        return result.lstrip()
    def expression(self, args):
        return args[0]
    def start(self, args):
        return args[1]

def parseInput(PQL):
    length = len(PQL)
    i = 0
    while(i < length):
        if(PQL[i] == ","):
            PQL = combineMultiple(PQL, i)
        i = i + 1 
    return PQL


def optimadeToMongoDBConverter(optimadeQuery):
    p = Parser(version=(0, 9, 5))
    print("optimadeQuery:", optimadeQuery)
    print()
    tree = p.parse(optimadeQuery)
    print("tree:\n", tree)
    print()
    rawPQL = OptimadeToPQLTransformer().transform(tree)
    print("rawPQL:\n", rawPQL)
    print()
    cleanedPQL = parseInput(rawPQL)
    return pql.find(cleanedPQL)
    

In [66]:
# sampleInput1 = 'filter=formula_prototype="AB2C3" and nelements=4 and elements="Si, O"'
# sampleInput2 = "filter=formula_prototype='AB2C3' or nelements=4 or elements='Si, O'"
sampleInput3 = 'filter=formula_prototype="AB2C3" and nelements=4 or elements="Si, O"'
# print("input1:", optimadeToMongoDBConverter(sampleInput1))
# print("result:\n", optimadeToMongoDBConverter(sampleInput2))
print("input3", optimadeToMongoDBConverter(sampleInput3))

optimadeQuery: filter=formula_prototype="AB2C3" and nelements=4 or elements="Si, O"

tree:
 Tree(start, [Token(KEYWORD, 'filter='), Tree(expression, [Tree(expression, [Tree(term, [Tree(term, [Tree(atom, [Tree(comparison, [Token(VALUE, 'formula_prototype'), Token(OPERATOR, '='), Token(VALUE, '"AB2C3"')])])]), Token(AND, 'and'), Tree(atom, [Tree(comparison, [Token(VALUE, 'nelements'), Token(OPERATOR, '='), Token(VALUE, '4')])])])]), Token(OR, 'or'), Tree(term, [Tree(atom, [Tree(comparison, [Token(VALUE, 'elements'), Token(OPERATOR, '='), Token(VALUE, '"Si, O"')])])])])])

rawPQL:
 formula_prototype=="AB2C3" and nelements==4

input3 {'$and': [{'formula_prototype': 'AB2C3'}, {'nelements': 4}]}


In [18]:
def removeFilter(raw):
    if(raw.find("filter =") != -1):
        return raw[len("filter ="):]
    elif(raw.find("filter=") != -1):
        return raw[len("filter="):]
    else:
        return raw
    
def insert(original, index, insertion):
    result = original[:index] + insertion + original[index:]
    return result

def onlyEqual(string, index):
    return string[index] == "=" and not (string[index-1] == "<" 
                     or string[index+1] == ">" 
                     or string[index-1] == "=" 
                     or string[index+1] == "=") 


def combineMultiple(string, index):
    firstQuoteIndex = 0
    for firstQuoteIndex in reversed(range(0, index)):
        if(ord(string[firstQuoteIndex]) == 34):
            break
    lastQuoteIndex = index
    for lastQuoteIndex in range(index, len(string)):
        if(ord(string[lastQuoteIndex])== 34):
            break
    insertion = string[firstQuoteIndex+1:lastQuoteIndex]
    insertion = insertion.split(",")
    return string[:firstQuoteIndex] + 'all({})'.format(insertion) + string[lastQuoteIndex+1:]